<a href="https://colab.research.google.com/github/Saim-Hassan786/Learn-Agentic-AI-With-OpenAI-Agents-SDK/blob/main/07-Tracing/Tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tracing
Tracing is an inbuilt feature iven by the OpenAI Agents SDK that is used for logging and debuging of entier agent run , tracing is divided into to types:

1. **Trace**: this is a top-level recoed of whole agent workflow or run
2. **Span** : this includes a timed step in a trace that logs a single specific operation like LLM response, tool call and handoffs etc.  

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 757.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('GOOGLE_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key = GOOGLE_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")

# Simple Tracing Setup
This is the most simple one and this tracing can be done by using only OpenAI API Keys and can be traced on OpenAI Platform only and these tracings are exported there.
To enable this , made following changes in above cell code:

1. **Replace Gemini API Key With OpenAI API Key**
2. **Replace Gemini Model With OpenAI Model**

In [ ]:
from agents import Agent,Runner,set_tracing_disabled,trace
set_tracing_disabled(False)

agent_with_tracing = Agent(
    name="Tracing Agent",
    instructions="A simple assistant agent",
    model = "gemini-2.5-flash"
)

# tracing setup
with trace("Simple Tracing"):
  result_with_tracing = await Runner.run(
    agent_with_tracing,
    "Hi there , myself Saim?"
)
  print(result_with_tracing.final_output)

Hi Saim! Nice to meet you. How can I help you today?


# Tracing to view Locally can also be called a Local Tracing

In [ ]:
from threading import local
from agents import Agent,Runner,set_trace_processors,set_default_openai_client,set_default_openai_api,set_trace_processors,trace
from openai import AsyncOpenAI
from agents.tracing.processor_interface import TracingProcessor

external_client_2 =  AsyncOpenAI(
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key = GOOGLE_API_KEY
)
set_default_openai_client(
    client=external_client_2,
    use_for_tracing=True
)
set_default_openai_api("chat_completions")

# tracing
class LocalTracing(TracingProcessor):
  def __init__(self):
    self.traces = []
    self.spans = []

  def on_trace_start(self, trace):
    self.traces.append(trace)
    print("========Trace Started========")
    print(f"Trace Name = {trace.name}")

  def on_trace_end(self, trace):
    print("========Trace Ended========")
    print(f"Trace Name = {trace.name}")
    print("******TRACE ENDED******")
    print("======TRACE DETAILS======")
    print(f"{trace.export()}")

  def on_span_start(self, span) :
    self.spans.append(span)
    print("========Span Started========")
    print(f"Span Id = {span.span_id}")
    print("======SPAN DETAILS======")
    print(span.export())

  def on_span_end(self, span):
    print("========Span Ended========")
    print(f"Span Id = {span.span_id}")
    print("******SPAN ENDED******")
    print("======SPAN DETAILS======")
    print(span.export())

  def force_flush(self) :
   pass    # use this to flush tracing into some data file etc , have to implement thi here as it is an abstract method of TracinProcessor Cls

  def shutdown(self):
    print("=======SHUTTING DOWN LOCAL TRACING========")
    print("*************Collected Traces****************")
    for trace in self.traces:
      print(trace.export())
    print("*************Collected Spans****************")
    for span in self.spans:
      print(span.export())

# local tracing setup
local_tracing_processor = LocalTracing()
set_trace_processors([local_tracing_processor])

# now agent setup

agent_with_local_tracing = Agent(
    name = "Local Tracing Agent",
    instructions = "A simple assistant agent",
    model = "gemini-2.5-flash"
)

with trace("Local Tracing"):
  result_with_local_tracing = await Runner.run(
    agent_with_local_tracing,
    "Hi there , myself Saim?"
  )
  print("************Final Output*************")
  print(result_with_local_tracing.final_output)

========Trace Started========
Trace Name = Local Tracing
========Span Started========
Span Id = span_c4dbae70b37948d78fadcbd7
======SPAN DETAILS======
{'object': 'trace.span', 'id': 'span_c4dbae70b37948d78fadcbd7', 'trace_id': 'trace_64109261d0cf4ea3950368f8e672d2ff', 'parent_id': None, 'started_at': '2025-07-09T11:27:11.418253+00:00', 'ended_at': None, 'span_data': {'type': 'agent', 'name': 'Local Tracing Agent', 'handoffs': [], 'tools': None, 'output_type': 'str'}, 'error': None}
========Span Started========
Span Id = span_cee28fd9fa3d4929a53b3ff8
======SPAN DETAILS======
{'object': 'trace.span', 'id': 'span_cee28fd9fa3d4929a53b3ff8', 'trace_id': 'trace_64109261d0cf4ea3950368f8e672d2ff', 'parent_id': 'span_c4dbae70b37948d78fadcbd7', 'started_at': '2025-07-09T11:27:11.419039+00:00', 'ended_at': None, 'span_data': {'type': 'generation', 'input': None, 'output': None, 'model': 'gemini-2.5-flash', 'model_config': {'temperature': None, 'top_p': None, 'frequency_penalty': None, 'presence_p

**To execute the shutdown() method in our local tracing processor , we have to shutdown our processor**

In [ ]:
local_tracing_processor.shutdown()

=======SHUTTING DOWN LOCAL TRACING========
*************Collected Traces****************
{'object': 'trace', 'id': 'trace_64109261d0cf4ea3950368f8e672d2ff', 'workflow_name': 'Local Tracing', 'group_id': None, 'metadata': None}
*************Collected Spans****************
{'object': 'trace.span', 'id': 'span_c4dbae70b37948d78fadcbd7', 'trace_id': 'trace_64109261d0cf4ea3950368f8e672d2ff', 'parent_id': None, 'started_at': '2025-07-09T11:27:11.418253+00:00', 'ended_at': '2025-07-09T11:27:12.159641+00:00', 'span_data': {'type': 'agent', 'name': 'Local Tracing Agent', 'handoffs': [], 'tools': [], 'output_type': 'str'}, 'error': None}
{'object': 'trace.span', 'id': 'span_cee28fd9fa3d4929a53b3ff8', 'trace_id': 'trace_64109261d0cf4ea3950368f8e672d2ff', 'parent_id': 'span_c4dbae70b37948d78fadcbd7', 'started_at': '2025-07-09T11:27:11.419039+00:00', 'ended_at': '2025-07-09T11:27:12.158486+00:00', 'span_data': {'type': 'generation', 'input': [{'content': 'A simple assistant agent', 'role': 'system'

# Tracing With 3rd Party Tracing Processor

**We can connect different 3rd party tracing processors to catch , save and display our traces of the Agent Run for us , these different tracing processors are :**

1. **Braintrust**
2. **Pydantic Logfire**
3. **AgentOps**
4. **Scorecard**
5. **Keywords AI**
6. **LangSmith**
7. **Maxim AI**
8. **Comet Opik**
9. **Langfuse**
10. **Langtrace**

**And many more......**

# **For example , I will implement AgentOps here below**

In [ ]:
!pip install -Uq agentops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.1/279.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 14.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
AGENTOPS_API_KEY = userdata.get("AGENTOPS_API_KEY_2")

In [ ]:
import agentops
agentops.init(AGENTOPS_API_KEY)

🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: AgentOps: Successfully instrumented 'OpenaiInstrumentor' for package 'agentops.instrumentation.providers.openai'.
🖇 AgentOps: AgentOps: 'agents' is the first-targeted agentic library. Will uninstrument providers if any are/become active.
🖇 AgentOps: AgentOps: Uninstrumented provider: OpenaiInstrumentor (for package 'openai') due to agentic library activation.
🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: AgentOps: Successfully instrumented 'OpenaiInstrumentor' for package 'agentops.instrumentation.providers.openai'.
🖇 AgentOps: AgentOps: Successfully instrumented 'OpenAIAgentsInstrumentor' for package 'agentops.instrumentation.agentic.openai_agents'.


In [ ]:
import asyncio
async def main():
   agent = Agent(
    name = "Joke Teller Agent",
    instructions="Tell a joke to the user",
    model = "gemini-2.0-flash"
)

   with trace("Joker Tracing"):
    result_1 = await Runner.run(agent, "Tell me a joke")
    result_2 = await Runner.run(agent, f"Rate this joke out of 10 {result_1.final_output}")
    print(result_1.final_output)
    print(result_2.final_output)

asyncio.run(main())

Why don't scientists trust atoms?

Because they make up everything!

Okay, here's my rating and why:

*   **Rating: 7/10**

Here's my reasoning:

*   **The Setup:** The setup is classic and familiar. It poses a question that makes you think there might be a complicated or science-specific reason.
*   **The Punchline:** The punchline is simple, punny, and relatable. Everyone understands that atoms are the building blocks of matter, and the play on words is effective.
*   **Overall:** It's a clean, easy-to-understand joke that gets a chuckle. It's not mind-blowing, but it's solid and works well as a quick, lighthearted joke.



# **All the Traces and Spans are sent directly to the AgentOps DashBoard and can be Observed from there .**